# Data preparation and infrastructure exposure to flooding

This notebook forms the basis of "Hands-On 5" in the CCG course.

1. Extract infrastructure data from OpenStreetMap
2. Extract flood hazard data from Aqueduct
3. Intersect floods with roads to calculate exposure
4. Open QGIS to look at the data

## Activity 1: Extract infrastructure data

### Step 1) On your desktop, create a folder called `ghana_tutorial`

### Step 2) Create a variable to store the folder location

In the cell below, type in the path to your desktop, by changing NAME to match your username as shown on your computer

In [ ]:
# edit this if using a Mac (otherwise delete)
data_folder = "/Users/NAME/Desktop/ghana_tutorial"

# edit this if using Windows (otherwise delete)
data_folder = "C:\\Users\\NAME\\Desktop\\ghana_tutorial"

### Step 3) Load Python libraries

In [ ]:
# The os and subprocess modules are built into Python
# see https://docs.python.org/3/library/os.html
import os 
# see https://docs.python.org/3/library/subprocess.html
import subprocess 

# Pandas and GeoPandas are libraries for working with datasets
# see https://geopandas.org/
import geopandas as gpd
# see https://pandas.pydata.org/
import pandas as pd 

# PyPROJ is a library for working with geographic projections 
# see https://pyproj4.github.io/
from pyproj import Geod

### Step 4) and 5) Download and save data

Download the `ghana-latest-free.shp.zip` dataset from http://download.geofabrik.de/africa/ghana.html, extract the zip folder and save the extracted folder within your new folder `ghana_tutorial`

### Step 6) Load the road dataset you've just downloaded

In [ ]:
GHA_OSM_roads = gpd.read_file(
    os.path.join(data_folder, 'ghana-latest-free.shp', 'gis_osm_roads_free_1.shp'))

### Step 7) To take a look at the data and the attribute table fill in and run the next two cells

In [ ]:
GHA_OSM_roads

In [ ]:
GHA_OSM_roads.fclass.unique()

### Step 8) Next we want to make a couple of changes to the data

In [ ]:
# Keep only the specified columns
GHA_OSM_roads = GHA_OSM_roads[['osm_id', 'fclass', 'name', 'geometry']]
# Keep only the roads whose "fclass" is in the list
GHA_OSM_roads = GHA_OSM_roads[
    GHA_OSM_roads.fclass.isin([
        'motorway',
        'motorway_link',
        'trunk',
        'trunk_link',
        'primary', 
        'primary_link',
        'secondary', 
        'secondary_link', 
        'tertiary',
        'tertiary_link'
    ])
]
# Reset the index numbering
GHA_OSM_roads = GHA_OSM_roads.reset_index(drop=True).reset_index()
# Rename some columns
GHA_OSM_roads = GHA_OSM_roads.rename(
    columns={
        'index': 'id',
        'fclass': 'road_type',
    })

Calculate the length of each road segment in meters

In [ ]:
geod = Geod(ellps='WGS84')
GHA_OSM_roads['length_m'] = GHA_OSM_roads.geometry.apply(geod.geometry_length)

In [ ]:
GHA_OSM_roads.tail()

### Step 9) Save the pre-processed dataset

In [ ]:
GHA_OSM_roads.to_file(
    os.path.join(data_folder, 'GHA_OSM_roads.gpkg'),
    layer='OSM-roads', 
    driver="GPKG")

## Activity 2: Extract and polygonise hazard data

### Step 1) Download a few tif files from Aqueduct



Save all downloaded tif files in a new folder titled `flood_layer` under your data_folder

### Step 2) Run the code below to polygonise the tif files

This converts the flood maps from *tiff files (raster data)* into *shape files (vector data)*. It will take a little time to run.

In [ ]:
xmin = "-3.262509"
ymin = "4.737128"
xmax = "1.187968"
ymax = "11.162937"

for root, dirs, files in os.walk(data_folder, 'flood_layer'): 
    print("Looking in", root) 
    for file in sorted(files): 
        if file.endswith(".tif") and not file.endswith("m.tif"): 
            print("Found tif file", file)
            stem = file[:-4]
            input_file = os.path.join(root, file) 
            
            # Clip file to bounds
            clip_file = os.path.join(root, f"{stem}_clipm.tif")
            try:
                os.remove(clip_file)
            except FileNotFoundError:
                pass
            p = subprocess.run([
                "gdalwarp", "-te", xmin, ymin, xmax, ymax, input_file, clip_file],
                capture_output=True)
            print(p.stdout.decode('utf8'))
            print(p.stderr.decode('utf8'))
            print(clip_file)
            
            for min_depth in ("1.0", "2.0"):
                # Create binary raster at threshold
                threshold_file = os.path.join(root, f"{stem}_{min_depth}m999.0m.tif")
                p = subprocess.run([
                    "gdal_calc.py",
                    "-A", clip_file,
                    f"--outfile={threshold_file}",
                    f"--calc=A>={min_depth}",
                    "--type=Byte",
                    "--NoDataValue=0",
                    "--co=SPARSE_OK=YES",
                    "--co=NBITS=1",
                    "--quiet",
                    "--co=COMPRESS=LZW"],
                    capture_output=True)
                print(p.stdout.decode('utf8'))
                print(p.stderr.decode('utf8'))
                print(threshold_file)
                
                # Create vector outline of raster areas
                polygons_file = os.path.join(root, f"{stem}_{min_depth}m999.0m.gpkg") 
                try:
                    os.remove(polygons_file)
                except FileNotFoundError:
                    pass
                p = subprocess.run([
                    "gdal_polygonize.py", threshold_file,'-q','-f', 'GPKG', polygons_file],
                    capture_output=True)
                print(p.stdout.decode('utf8'))
                print(p.stderr.decode('utf8'))  
                print(polygons_file)

## Activity 3: Intersect hazard 

Let us now intersect the hazard and the roads, starting with one hazard initially so we save time.

### Step 1) Specify your input and output path as well as the name of the intersection

In [ ]:
flood_path = os.path.join(
    data_folder, 
    'flood_layer', 
    'inunriver_historical_000000000WATCH_1980_rp00010_1.0m999.0m.gpkg')

output_path = os.path.join(
    data_folder, 
    'results', 
    'inunriver_historical_000000000WATCH_1980_rp00010_1.0m999.0m_exposure.gpkg')

flood_gpd = gpd.read_file(flood_path)

### Step 2) Run the intersection

In [ ]:
flood_intersections = gpd.overlay(GHA_OSM_roads, flood_gpd, how='intersection')
# flood_intersections = gpd.clip(GHA_OSM_roads, flood_gpd, keep_geom_type=True)

Calculate the exposed length

In [ ]:
geod = Geod(ellps='WGS84')
flood_intersections['flood_length_m'] = flood_intersections.geometry.apply(geod.geometry_length)

In [ ]:
GHA_OSM_roads[GHA_OSM_roads.osm_id == "863568484"]

In [ ]:
flood_intersections.tail()

Calculate the proportion of roads in our dataset which are exposed to >=1m flood depths in this scenario

In [ ]:
exposed_length = flood_intersections.flood_length_m.sum()

In [ ]:
all_roads_in_dataset_length = GHA_OSM_roads.length_m.sum()

In [ ]:
proportion = exposed_length / all_roads_in_dataset_length
proportion

In [ ]:
f"{proportion:.0%} of roads in this dataset are exposed to flood depths of >= 1m in a historical 1-in-10 year flood"

Save to file (with spatial data)

In [ ]:
flood_intersections.to_file(output_path, driver="GPKG")

In [ ]:
flood_intersections

Save to CSV (without spatial data)

In [ ]:
flood_intersections.drop(columns='geometry').to_csv(output_path.replace(".gpkg", ".csv"))